<a href="https://colab.research.google.com/github/smgchris/ncsa-facelogin/blob/main/ncsa_face_detection_login.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install flask-ngrok

In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask

app=Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
  return "<h1>This is home</h1>"

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9fb7ae1b912f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install flask_cors

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import os
import cv2
import json
import numpy as np
from flask_cors import CORS
from tensorflow.python.keras.backend import set_session
import base64
from datetime import datetime

tf.compat.v1.disable_eager_execution()

graph = tf.compat.v1.get_default_graph()

app=Flask(__name__)
run_with_ngrok(app)

CORS(app)

sess = tf.compat.v1.Session()

set_session(sess)

model = tf.keras.models.load_model('/content/drive/MyDrive/keras-facenet/model/facenet_keras.h5')


def img_to_encoding(path, model):
  img1 = cv2.imread(path, 1)
  img = img1[...,::-1]
  dim = (160, 160)
  # resize image
  if(img.shape != (160, 160, 3)):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  x_train = np.array([img])
  print(x_train)
  embedding = model.predict(x_train)
  #print("embedding",embedding)
  return embedding

database = {}

def verify(image_path, identity, database, model):
  
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding-database[identity])
    print(dist)
    if dist<5:
        print("It's " + str(identity) + ", welcome in!")
        match = True
    else:
        print("It's not " + str(identity) + ", please go away")
        match = False
    return dist, match

@app.route('/register', methods=['POST'])
def register():
    try:
        username = request.get_json()['username']
        img_data = request.get_json()['image64']
        with open('/content/drive/MyDrive/ncsa/images/'+username+'.jpg', "wb") as fh:
            fh.write(base64.b64decode(img_data[22:]))
        path = '/content/drive/MyDrive/ncsa/images/'+username+'.jpg'

        global sess
        global graph
        with graph.as_default():
            set_session(sess)
            database[username] = img_to_encoding(path, model)    
        return json.dumps({"status": 200})
    except:
        return json.dumps({"status": 500})

def who_is_it(image_path, database, model):
    # print(image_path)
    encoding = img_to_encoding(image_path, model)
    min_dist = 1000
    identity="anonymous"
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding-db_enc)
        print(dist)
        if dist<min_dist:
            min_dist = dist
            identity = name
    if min_dist > 5:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    return min_dist, identity

@app.route('/verify', methods=['POST'])
def change():
    img_data = request.get_json()['image64']
    img_name = str(int(datetime.timestamp(datetime.now())))
    with open('/content/drive/MyDrive/ncsa/images/'+img_name+'.jpg', "wb") as fh:
        fh.write(base64.b64decode(img_data[22:]))
    path = '/content/drive/MyDrive/ncsa/images/'+img_name+'.jpg'
    global sess
    global graph
    with graph.as_default():
        set_session(sess)
        min_dist, identity = who_is_it(path, database, model)
    os.remove(path)
    if min_dist > 5:
        return json.dumps({"identity": 0})
    return json.dumps({"identity": str(identity),"distance": str(min_dist)})


app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c50055f3561b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
